# 네이버 증권 개별종목 부가정보

* 종목분석에 있는 기업개요의 업종정보를 가져오겠습니다. 

In [36]:
import requests 
from bs4 import BeautifulSoup
import re
import sqlite3
import pandas as pd
import time

## 사용할 DB를 호출합니다.

In [37]:
con = sqlite3.connect("./common_stock.db")

In [38]:
cur = con.cursor()

* 네이버 종목분석 탭에서 크롬을 사용하여 온라인 기업 정보 url 주소만 추출 

## 1. 종목의 업종을 저장합니다.

In [39]:
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}"

In [40]:
test_url = url.format("005930")

In [41]:
r = requests.get(test_url)

In [42]:
bs = BeautifulSoup(r.text, 'lxml')

In [43]:
stock_info = bs.find("td", {"class":"cmp-table-cell td0101"})

In [47]:
bs.find("td", class_ ="cmp-table-cell td0101")

<td class="cmp-table-cell td0101">
<dl>
<dt>
<span class="name">삼성전자</span>
<a class="cEm" href="http://www.sec.co.kr" target="comPage" title="[홈페이지] www.sec.co.kr"><img align="absmiddle" hspace="3" src="/images/icon_home.gif"/></a>
<a class="cEm" title="[대표전화] 031-200-1114       
[주식담당] 02-2255-9000"><img align="absmiddle" hspace="1" src="/images/icon_tel.gif"/></a>
<b class="num">005930</b>
</dt>
<dt class="line-left">SamsungElec</dt>
<dt class="line-left">KOSPI : 전기전자</dt>
<dt class="line-left">WICS : 반도체와반도체장비</dt>
</dl>
</td>

In [44]:
stock_info

<td class="cmp-table-cell td0101">
<dl>
<dt>
<span class="name">삼성전자</span>
<a class="cEm" href="http://www.sec.co.kr" target="comPage" title="[홈페이지] www.sec.co.kr"><img align="absmiddle" hspace="3" src="/images/icon_home.gif"/></a>
<a class="cEm" title="[대표전화] 031-200-1114       
[주식담당] 02-2255-9000"><img align="absmiddle" hspace="1" src="/images/icon_tel.gif"/></a>
<b class="num">005930</b>
</dt>
<dt class="line-left">SamsungElec</dt>
<dt class="line-left">KOSPI : 전기전자</dt>
<dt class="line-left">WICS : 반도체와반도체장비</dt>
</dl>
</td>

In [50]:
(stock_info.find_all(['span', "dt"], {"class" : ["name", "line-left"]}))

[<span class="name">삼성전자</span>,
 <dt class="line-left">SamsungElec</dt>,
 <dt class="line-left">KOSPI : 전기전자</dt>,
 <dt class="line-left">WICS : 반도체와반도체장비</dt>]

In [51]:
[x.text.split(":")[-1].strip() for x in stock_info.find_all(['span', "dt"], {"class" : ["name", "line-left"]})]

['삼성전자', 'SamsungElec', '전기전자', '반도체와반도체장비']

In [12]:
stock_code = cur.execute("SELECT * FROM MASTER").fetchall()

In [13]:
stock_code

[('가비아', 'KR7079940003', 'KOSDAQ', '079940'),
 ('가온미디어', 'KR7078890001', 'KOSDAQ', '078890'),
 ('가온전선', 'KR7000500009', 'KOSPI', '000500'),
 ('감마누', 'KR7192410009', 'KOSDAQ', '192410'),
 ('강남제비스코', 'KR7000860007', 'KOSPI', '000860'),
 ('강스템바이오텍', 'KR7217730001', 'KOSDAQ', '217730'),
 ('강원랜드', 'KR7035250000', 'KOSPI', '035250'),
 ('갤럭시아에스엠', 'KR7011420007', 'KOSPI', '011420'),
 ('갤럭시아컴즈', 'KR7094480001', 'KOSDAQ', '094480'),
 ('게임빌', 'KR7063080006', 'KOSDAQ', '063080'),
 ('경남스틸', 'KR7039240007', 'KOSDAQ', '039240'),
 ('경남제약', 'KR7053950002', 'KOSDAQ', '053950'),
 ('경농', 'KR7002100006', 'KOSPI', '002100'),
 ('경동나비엔', 'KR7009450008', 'KOSPI', '009450'),
 ('경동도시가스', 'KR7267290005', 'KOSPI', '267290'),
 ('경동인베스트', 'KR7012320008', 'KOSPI', '012320'),
 ('경동제약', 'KR7011040003', 'KOSDAQ', '011040'),
 ('경방', 'KR7000050005', 'KOSPI', '000050'),
 ('경보제약', 'KR7214390007', 'KOSPI', '214390'),
 ('경인양행', 'KR7012610002', 'KOSPI', '012610'),
 ('경인전자', 'KR7009140005', 'KOSPI', '009140'),
 ('경창산업', 'KR702

### 테이블을 설계해서 입력하겠습니다.

In [52]:
query = """CREATE TABLE IF NOT EXISTS "stock_code_type" (
    "종목코드" TEXT NOT NULL,
    "종목이름" TEXT NOT NULL,
    "영문"     TEXT NOT NULL,
    "KOSPI"    TEXT,
    "WICS"     TEXT,
    PRIMARY KEY('종목코드')
)"""

In [55]:
cur.execute(query)

In [25]:
total = []
for name in stock_code:
    print(name[-1])
    # 접속 오류가 발생되면 다시 실행하기 위해서 
    while(True):
        try:
            r = requests.get(url.format(name[-1]))
            break
        except Exception as e:
            print (e)
            time.slee(3)
        
    bs = BeautifulSoup(r.text, 'lxml')
    try:
        stock_info = bs.find("td", {"class":"cmp-table-cell td0101"})
        rt = [x.text.split(":")[-1] for x in stock_info.find_all(['span', "dt"], {"class" : ["name", "line-left"]})]
        # 코드명을 추가한다
        rt.insert(0,name[-1])
        total.append(rt)
    except Exception as e :
        print(name, e)

079940
078890
000500
192410
000860
217730
035250
011420
094480
063080
039240
053950
002100
009450
267290
012320
011040
000050
214390
012610
009140
024910
013580
012200
012205
004200
002140
198440
049720
010130
002240
014570
098460
038530
900280
215000
121440
183410
('골프존데카', 'KR7183410000', 'KONEX', '183410') 'NoneType' object has no attribute 'find_all'
076340
('관악산업', 'KR7076340009', 'KONEX', '076340') 'NoneType' object has no attribute 'find_all'
009290
014200
017040
017900
037710
026910
090150
267320
307280
030610
053270
007690
005320
001140
066620
043650
006050
060480
078130
002720
243870
HTTPSConnectionPool(host='navercomp.wisereport.co.kr', port=443): Max retries exceeded with url: /v2/company/c1010001.aspx?cmp_cd=243870 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))


From cffi callback <function _verify_callback at 0x00000232AB95D488>:
Traceback (most recent call last):
  File "C:\Users\woong\Anaconda3\envs\rpy\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


AttributeError: module 'time' has no attribute 'slee'

In [26]:
total_df = pd.DataFrame(total)

In [27]:
total_df.columns = ["종목코드", "종목이름", "영문",  "KOSPI", "WICS"]

In [28]:
total_df.head(3)

,종목코드,종목이름,영문,KOSPI,WICS
0,079940,가비아,GABIA,인터넷,IT서비스
1,078890,가온미디어,Kaon Media,통신장비,전자제품
2,000500,가온전선,GAONCABLE,전기전자,전기장비


In [29]:
total_df.to_sql("stock_code_type", con=con, if_exists='append', index=False)

## konex 제외하기

In [30]:
stock_code = cur.execute("SELECT * FROM MASTER where marketName != 'KONEX'").fetchall()

In [31]:
len(stock_code)

2249

* for를 konex를 제외한 결과로 다시 실행합니다.

In [32]:
total = []
for name in stock_code:
    print(name[-1])
    # 접속 오류가 발생되면 다시 실행하기 위해서 
    while(True):
        try:
            r = requests.get(url.format(name[-1]))
            break
        except Exception as e:
            print (e)
            time.slee(3)
        
    bs = BeautifulSoup(r.text, 'lxml')
    try:
        stock_info = bs.find("td", {"class":"cmp-table-cell td0101"})
        rt = [x.text.split(":")[-1] for x in stock_info.find_all(['span', "dt"], {"class" : ["name", "line-left"]})]
        # 코드명을 추가한다
        rt.insert(0,name[-1])
        total.append(rt)
    except Exception as e :
        print(name, e)

079940
078890
000500
192410
000860
217730
035250
011420
094480
063080
039240
053950
002100
009450
267290
012320
011040
000050
214390
012610
009140
024910
013580
012200
012205
004200
002140
198440
049720
010130
002240
014570
098460
038530
900280
215000
121440
009290
014200
017040
017900
037710
026910
090150
267320
307280
030610
053270
007690
005320
001140
066620
043650
006050
060480
078130
002720
083420
014530
900070
204620
019660
014280
014285
053260
008870
001570
002990
002995
011780
011785
214330
001210
073240
036190
049080
035460
092440
000270
024110
013700
308100
004540
004545
187790
074610
151910
121600
039860
091970
051490
190510
242040
089600
293580
138610
130580
036800
001260
008350
008355
004270
003920
003925
002070
025860
111710
091590
168330
072770
253590
212560
095660
042420
085910


KeyboardInterrupt: 

In [33]:
total_df = pd.DataFrame(total)
total_df.columns = ["종목코드", "종목이름", "영문",  "KOSPI", "WICS"]

In [35]:
total_df.to_sql("stock_code_type", con=con, if_exists='replace', index=False)

## 2. 시세 및 주주현황

In [261]:
r= requests.get("https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930")
bs_rt = BeautifulSoup(r.text, 'html.parser')

In [262]:
find_table = bs_rt.find("table", id="cTB11")

In [263]:
find_table

<table class="gHead" id="cTB11" summary="기업의 기본적인 시세정보(주가/전일대비/수익률,52주최고/최저,액면가,거래량/거래대금,시가총액,유동주식비율,외국인지분율,52주베타,수익률(1M/3M/6M/1Y))를 제공합니다.">
<caption class="blind">시세정보</caption>
<colgroup>
<col width="125"/>
<col width="*"/>
</colgroup>
<tbody>
<tr>
<th class="txt" scope="row">주가/전일대비/수익률</th>
<td class="num"><strong>
															46,800</strong>원 /
														<span class="cUp">+700원</span> / <span class="cUp">+1.52%</span>
</td>
</tr>
<tr>
<th class="txt" scope="row">52Weeks 최고/최저</th>
<td class="num">
															48,450원 /
														36,850원
														</td>
</tr>
<tr>
<th class="txt" scope="row">액면가</th>
<td class="num">
															100원
														</td>
</tr>
<tr>
<th class="txt" scope="row">거래량/거래대금</th>
<td class="num">
															8,819,390주 /
														4,129억원
														</td>
</tr>
<tr>
<th class="txt" scope="row">시가총액</th>
<td class="num">
															2,793,858억원
														</td>
</tr>
<tr>
<th class="txt" scope="row">52주베타</th>
<td class

In [264]:
tr_data = find_table.findAll("tr")

In [77]:
for idx, x in enumerate(tr_data):
    print (idx, "---->" , x.find("td").text.strip())

0 ----> 46,800원 /
														+700원 / +1.52%
1 ----> 48,450원 /
														36,850원
2 ----> 100원
3 ----> 8,819,390주 /
														4,129억원
4 ----> 2,793,858억원
5 ----> 1.50
6 ----> 5,969,782,550주 / 74.46%
7 ----> 57.74%
8 ----> +3.20%/ +3.31%/ +10.64%/ -0.21%


In [78]:
x.find("td").text.strip().split('/')

['+3.20%', ' +3.31%', ' +10.64%', ' -0.21%']

* 기준일을 찾겠습니다. 

In [102]:
# 날짜 패턴을 만들어서 데이터를 가져옵니다. xxxx.xx.xx
day_pattern = re.compile("[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}")

In [265]:
day_text = bs_rt.find("dd", class_="header-table-cell unit").p.text

In [266]:
day_pattern.search(day_text).group()

'2019.07.19'

In [269]:
pd.read_html(str(find_table))[0]

,0,1
0,주가/전일대비/수익률,"46,800원 / +700원 / +1.52%"
1,52Weeks 최고/최저,"48,450원 / 36,850원"
2,액면가,100원
3,거래량/거래대금,"8,819,390주 / 4,129억원"
4,시가총액,"2,793,858억원"
5,52주베타,1.50
6,발행주식수/유동비율,"5,969,782,550주 / 74.46%"
7,외국인지분율,57.74%
8,수익률 (1M/3M/6M/1Y),+3.20%/ +3.31%/ +10.64%/ -0.21%


### for문으로 전체를 돌려보겠습니다.

In [103]:
total_sise = []
pattern = re.compile("[0-9,.\+-]+")
day_pattern = re.compile("[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}")
for name in stock_code:
    #print(name[-1])
    # 접속 오류가 발생되면 다시 실행하기 위해서 
    while(True):
        try:
            r = requests.get(url.format(name[-1]))
            break
        except Exception as e:
            print (e)
            print (r.url)
            time.slee(3)
        
    bs_rt = BeautifulSoup(r.text, 'lxml')
    try:
        tr_data = bs_rt.find("table", id="cTB11").findAll("tr")
        day = day_pattern.search(bs_rt.find("dd", class_="header-table-cell unit").p.text).group()
        sise = []
        for idx, x in enumerate(tr_data):
            sise.extend(pattern.findall(x.find("td").text))
        if(len(sise) != 17):
            print(name[-1])
            print("17개가 아니다")
            break
        sise.insert(0, name[-1])
        sise.insert(1, day)
        total_sise.append(sise)
    except Exception as e :
        print(name, e)
    

('현대정보기술', 'KR7026180000', 'KOSDAQ', '026180') 'NoneType' object has no attribute 'findAll'


In [104]:
total_sise

[['079940',
  '2019.07.19',
  '8,560',
  '+280',
  '+3.38',
  '10,750',
  '5,650',
  '500',
  '75,567',
  '6',
  '1,159',
  '1.12',
  '13,535,684',
  '71.51',
  '17.02',
  '-5.52',
  '-4.14',
  '+27.95',
  '-10.55'],
 ['078890',
  '2019.07.19',
  '10,250',
  '+200',
  '+1.99',
  '14,450',
  '4,905',
  '500',
  '593,169',
  '60',
  '1,515',
  '1.38',
  '14,784,585',
  '81.08',
  '5.14',
  '-21.76',
  '+3.33',
  '+20.45',
  '+39.08'],
 ['000500',
  '2019.07.19',
  '19,200',
  '+250',
  '+1.32',
  '25,150',
  '15,750',
  '5,000',
  '2,046',
  '0',
  '799',
  '0.68',
  '4,160,347',
  '56.78',
  '4.57',
  '-1.29',
  '+5.21',
  '+2.67',
  '-15.42'],
 ['192410',
  '2019.07.19',
  '408',
  '0',
  '0',
  '6,170',
  '408',
  '500',
  '0',
  '0',
  '100',
  '0.16',
  '24,435,800',
  '48.96',
  '0.29',
  '0',
  '0',
  '0',
  '-93.39'],
 ['000860',
  '2019.07.19',
  '25,000',
  '+50',
  '+0.20',
  '30,900',
  '22,800',
  '1,000',
  '6,301',
  '2',
  '1,625',
  '0.69',
  '6,500,000',
  '49.25',
  '1

In [106]:
sise = pd.DataFrame(total_sise, columns=['종목코드', '기준일', '주가', '전일대비', '수익률', '최고', '최저', '액면가', '거래량', '거래대금', '시가총액',
                                  '52주베타', '발행주식수', '유동비율', '외국인지분율', '1M', '3M', '6M', '1Y'])

In [107]:
sise

,종목코드,기준일,주가,전일대비,수익률,최고,최저,액면가,거래량,거래대금,시가총액,52주베타,발행주식수,유동비율,외국인지분율,1M,3M,6M,1Y
0,079940,2019.07.19,"8,560",+280,+3.38,"10,750","5,650",500,"75,567",6,"1,159",1.12,"13,535,684",71.51,17.02,-5.52,-4.14,+27.95,-10.55
1,078890,2019.07.19,"10,250",+200,+1.99,"14,450","4,905",500,"593,169",60,"1,515",1.38,"14,784,585",81.08,5.14,-21.76,+3.33,+20.45,+39.08
2,000500,2019.07.19,"19,200",+250,+1.32,"25,150","15,750","5,000","2,046",0,799,0.68,"4,160,347",56.78,4.57,-1.29,+5.21,+2.67,-15.42
3,192410,2019.07.19,408,0,0,"6,170",408,500,0,0,100,0.16,"24,435,800",48.96,0.29,0,0,0,-93.39
4,000860,2019.07.19,"25,000",+50,+0.20,"30,900","22,800","1,000","6,301",2,"1,625",0.69,"6,500,000",49.25,13.00,-3.85,-9.26,-3.29,-2.53
5,217730,2019.07.19,"14,800",+50,+0.34,"24,400","9,250",500,"118,128",18,"3,085",1.05,"20,844,437",82.85,2.46,-12.68,-32.11,-22.31,+48.00
6,035250,2019.07.19,"30,800",+150,+0.49,"34,550","25,600",500,"228,282",70,"65,894",0.33,"213,940,500",58.50,30.07,-1.12,-6.95,-4.20,+15.36
7,011420,2019.07.19,"2,430",+560,+29.95,"2,495","1,205",500,"4,597,110",101,669,1.32,"27,549,644",62.91,0.05,+20.00,+48.62,+40.87,+28.91
8,094480,2019.07.19,"3,420",+120,+3.64,"4,570","2,420",500,"56,385",2,"1,342",0.79,"39,229,838",38.06,0.93,-13.42,-4.07,+7.21,-11.17
9,063080,2019.07.19,"39,400",-950,-2.35,"63,100","37,400",500,"44,548",17,"2,599",0.61,"6,595,192",63.96,11.78,-2.60,-18.76,-27.31,-22.44


## 3.주가 / 상대수익률 

In [270]:
import json
import datetime, time

In [271]:
r= requests.get("https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001270")

In [272]:
r.text.find("encparam")

40868

In [273]:
r.text[40800:40999]

'001270\',\r\n\t\t\t\t\t\tgrp_cd: grp==10?"01":"03",\r\n                        encparam: \'ejcwT05qYW9pem1EbkZZV29aamFudz09\'\r\n\t\t\t\t\t}\r\n\t\t\t\t, function (res) {\r\n\t\t\t\t\tvar data = []\r\n\t\t\t\t\t_.each(res.dt, function(v, i'

In [274]:
enc_pattern = re.compile("encparam: '(.+)'")

In [275]:
enc = enc_pattern.search(r.text).group(1)

In [276]:
enc

'ejcwT05qYW9pem1EbkZZV29aamFudz09'

In [277]:
payload = {}

* flag : 1 -> 주가 / 거래대금
* flag : 2 -> 주가/상대수익률
* flag : 3 -> 외국인지분율 / 기관1개월누적순매수

In [278]:
payload['flag'] = 2

In [279]:
payload['cmp_cd'] = '001270'

In [280]:
payload['encparam'] = enc

In [281]:
graph_url = "https://navercomp.wisereport.co.kr/company/ajax/cF1001.aspx?"

In [282]:
r = requests.get(graph_url, params=payload)

In [283]:
r.text

'{"TRD_DT":[1501459200000.0,1504137600000.0,1506643200000.0,1509408000000.0,1512000000000.0,1514419200000.0,1517356800000.0,1519776000000.0,1522368000000.0,1525046400000.0,1527724800000.0,1530230400000.0,1532995200000.0,1535673600000.0,1538092800000.0,1540944000000.0,1543536000000.0,1545955200000.0,1548892800000.0,1551312000000.0,1553817600000.0,1556582400000.0,1559260800000.0,1561680000000.0,1563494400000.0],"D1_NM":"부국증권","D2_NM":"KOSPI대비","D3_NM":"증권대비","D1":[26300.0,27600.0,27100.0,27500.0,26600.0,25150.0,28100.0,28500.0,27150.0,24350.0,23900.0,22400.0,23950.0,22850.0,22950.0,21950.0,23850.0,22450.0,20900.0,21000.0,20800.0,21400.0,20300.0,21100.0,20900.0],"D2":[0.00,6.59,3.38,-0.46,-1.93,-7.07,0.03,7.34,1.44,-12.10,-9.97,-11.64,-4.46,-9.80,-10.26,-1.01,3.41,0.41,-12.30,-11.53,-10.01,-10.34,-7.79,-8.45,-7.70],"D3":[0.00,11.02,13.92,13.10,7.26,4.52,-3.38,10.99,8.00,-6.62,-6.68,-3.51,7.99,2.05,-0.60,10.65,9.60,8.50,-4.66,-2.80,-3.00,-4.78,-5.76,-10.11,-9.05],"TRD_QTY":[15071.0,4935.0,

In [285]:
type(json.loads(r.text))

dict

In [221]:
json.loads(r.text).keys()

dict_keys(['TRD_DT', 'D1_NM', 'D2_NM', 'D1', 'D2', 'TRD_ADJ_PRC'])

In [286]:
graph_1 = pd.DataFrame(json.loads(r.text))

In [287]:
graph_1

,TRD_DT,D1_NM,D2_NM,D3_NM,D1,D2,D3,TRD_QTY,TRD_QTY_TYP
0,1.501459e+12,부국증권,KOSPI대비,증권대비,26300.0,0.00,0.00,15071.0,R
1,1.504138e+12,부국증권,KOSPI대비,증권대비,27600.0,6.59,11.02,4935.0,R
2,1.506643e+12,부국증권,KOSPI대비,증권대비,27100.0,3.38,13.92,3119.0,R
3,1.509408e+12,부국증권,KOSPI대비,증권대비,27500.0,-0.46,13.10,1538.0,R
4,1.512000e+12,부국증권,KOSPI대비,증권대비,26600.0,-1.93,7.26,2979.0,R
5,1.514419e+12,부국증권,KOSPI대비,증권대비,25150.0,-7.07,4.52,5363.0,B
6,1.517357e+12,부국증권,KOSPI대비,증권대비,28100.0,0.03,-3.38,43626.0,R
7,1.519776e+12,부국증권,KOSPI대비,증권대비,28500.0,7.34,10.99,32662.0,R
8,1.522368e+12,부국증권,KOSPI대비,증권대비,27150.0,1.44,8.00,3351.0,R
9,1.525046e+12,부국증권,KOSPI대비,증권대비,24350.0,-12.10,-6.62,4966.0,B


* 제공된 시간 데이터는 1970년 1월 1일 00:00:00을 기준으로 경과한 시간을 초로 환산하여 정수로 나타내는 POSIX 시간
* 이 시간을 우리가 보는 시간으로 변경하기 위해서 fromtimestamp() 메소드를 사용합니다.

In [229]:
graph_1['TRD_DT']= graph_1['TRD_DT'].apply(lambda x : datetime.datetime.fromtimestamp(int(str(x)[:10]))  )

In [230]:
graph_1

,TRD_DT,D1_NM,D2_NM,D3_NM,D1,D2,D3,TRD_QTY,TRD_QTY_TYP
0,2017-07-31 09:00:00,부국증권,KOSPI대비,증권대비,26300.0,0.00,0.00,15071.0,R
1,2017-08-31 09:00:00,부국증권,KOSPI대비,증권대비,27600.0,6.59,11.02,4935.0,R
2,2017-09-29 09:00:00,부국증권,KOSPI대비,증권대비,27100.0,3.38,13.92,3119.0,R
3,2017-10-31 09:00:00,부국증권,KOSPI대비,증권대비,27500.0,-0.46,13.10,1538.0,R
4,2017-11-30 09:00:00,부국증권,KOSPI대비,증권대비,26600.0,-1.93,7.26,2979.0,R
5,2017-12-28 09:00:00,부국증권,KOSPI대비,증권대비,25150.0,-7.07,4.52,5363.0,B
6,2018-01-31 09:00:00,부국증권,KOSPI대비,증권대비,28100.0,0.03,-3.38,43626.0,R
7,2018-02-28 09:00:00,부국증권,KOSPI대비,증권대비,28500.0,7.34,10.99,32662.0,R
8,2018-03-30 09:00:00,부국증권,KOSPI대비,증권대비,27150.0,1.44,8.00,3351.0,R
9,2018-04-30 09:00:00,부국증권,KOSPI대비,증권대비,24350.0,-12.10,-6.62,4966.0,B


### 직접 해보기 : 모든 종목을 반복문으로 실행하여 데이터를 가져옵니다. 그리고 DB에 Insert를 해봅시다. 

## 4. 펀더멘털

In [257]:
r= requests.get("https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001270")

In [258]:
bs = BeautifulSoup(r.text, 'lxml')

In [288]:
fundamental_bs = bs.find("table", class_="gHead03").find_all("tr")

In [289]:
fundamental_bs

[<tr>
 <th style="padding-left:0; padding-right:0">주요지표</th>
 <th style="padding-left:0; padding-right:0">2018/12(A)</th>
 <th class="endLine" style="padding-left:0; padding-right:0">2019/12(E)</th>
 </tr>, <tr>
 <th class="left" scope="row">PER</th>
 <td class="num">9.63</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">PBR</th>
 <td class="num">0.40</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">PCR</th>
 <td class="num">581.14</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">EV/EBITDA</th>
 <td class="num">12.30</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">EPS</th>
 <td class="num">2,170원</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">BPS</th>
 <td class="num">51,862원</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">EBITDA</th>
 <td class="num">414.5억원</td>
 <td class="

In [292]:
pd.read_html(str(bs.find("table", class_="gHead03")))[0]

,주요지표,2018/12(A),2019/12(E)
0,PER,9.63,NaN
1,PBR,0.40,NaN
2,PCR,581.14,NaN
3,EV/EBITDA,12.30,NaN
4,EPS,"2,170원",NaN
5,BPS,"51,862원",NaN
6,EBITDA,414.5억원,NaN
7,현금DPS,"1,200원",NaN
8,현금배당수익률,5.74%,NaN
9,회계기준,연결,연결
